# My College Basketball Data

In [34]:
#import
import csv
import numpy as np
import torch 

In [35]:
#set up dictionary for team number and team name
teamidentities = {}
with open('Teams.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    
    for row in readCSV:
        teamidentities.update( {row[0]: row[1]})
    
    #deletes first key 
    del teamidentities["Team_Id"]
    
    #print(teamidentities)

In [36]:
#read and data split tourney seeds
#1985-2017

trainingDataSetSeeds = []
testDataSetSeeds = []
validationDataSetSeeds = []

with open('TourneySeeds.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
   
    for row in readCSV:
        if row[0] == "2016":
            testDataSetSeeds.append(row)
        elif row[0] == "2017":
            validationDataSetSeeds.append(row)
        else:
            trainingDataSetSeeds.append(row)
            
#     print(trainingDataSetSeeds)
#     print(testDataSetSeeds)
#     print(validationDataSetSeeds)

In [37]:
#read and data split regular season stats
#2003-2015

trainingDataSet = []
testDataSet = []
validationDataSet = []

def dataSplit():
    for row in readCSV:
        if row[0] == "2014":
            testDataSet.append(row)
        elif row[0] == "2015":
            validationDataSet.append(row)
        else:
            trainingDataSet.append(row)
            
with open('RegularSeasonDetailedResults.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    
    dataSplit()

In [38]:
#read and data split tournament stats
#2003-2016

remainingDataSet = []
with open('TourneyDetailedResults.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    
    dataSplit()

#used dataSplit() to assemble trainingDataSet, testDataSet, validationDataSet by combining data from RegularSeasonedDetailedResults.csv and TourneyDetailedResults
#trainingDataSet made up of data from RSDR years of 2003-2013 and TDR years 2003-2013, 2016
#testDataSet made up of data from RSDR years 2014 and TDR years 2014
#validationDataSet made up of data from RSDR years 2015 and TDR years 2015

# print(trainingDataSet)
# print(testDataSet)
# print(validationDataSet)


In [39]:
#extracting features for each DataSet
#first half of the feature vector is winning team stats and last half is losing team stats
#features taken out: [Wteam, Wfgm, Wfga, Wfgm3, Wfga3, Wftm, Wfta, Wor, Wdr, Wast, Wto, Wstl, Wblk, Wpf, Lteam, Lfgm, Lfga, Lfgm3, Lfga3, Lftm, Lfta, Lor, Ldr, Last, Lto, Lstl, Lblk, Lpf]
#score is taken out

#function to featureExtract
def featureExtract(dataSetArray):
    featureExtractArray = []
    for row in dataSetArray:
        featureExtractArray.append([row[2], row[8], row[9], row[10], row[11], row[12], row[13], row[14], row[15], row[16], row[17], row[18], row[19], row[20], row[4], row[21], row[22], row[23], row[24], row[25], row[26], row[27], row[28], row[29], row[30], row[31] ] )
    return featureExtractArray[1:].copy()
    #print(featureExtractArray)
    
#trainingDataSet feature extraction
featuretrainingDataSet = featureExtract(trainingDataSet)
labeltrainingDataSet = [1] * len(featuretrainingDataSet)
#print(labeltrainingDataSet)

#testDataSet feature extraction
featuretestDataSet = featureExtract(testDataSet)
labeltestDataSet = [1] * len(featuretestDataSet)

#validationDataSet feature extraction
featurevalidationDataSet = featureExtract(validationDataSet)
labelvalidationDataSet = [1] * len(featurevalidationDataSet)

In [40]:
print(featuretrainingDataSet[2])

['1266', '24', '58', '8', '18', '17', '29', '17', '26', '15', '10', '5', '2', '25', '1437', '22', '73', '3', '26', '14', '23', '31', '22', '9', '12', '2']


In [52]:
#randomization of winning and losing team statistics from feature trainingDataSet
#new label
randomizationSeedTrainingSet = np.random.randint(2, size = len(featuretrainingDataSet))
#26 elements in featuretrainingDataSet
#print(randomizationSeedTrainingSet)

# randomization of winning and losing team statistics from feature testDataSet
randomizationSeedTestSet = np.random.randint(2, size = len(featuretestDataSet))

#randomization of winning and losing team statistics from feature validationDataSet
randomizationSeedValidationSet = np.random.randint(2, size = len(featurevalidationDataSet))

n_feat = len(featuretrainingDataSet[0])
assert n_feat % 2 == 0

#define as function
def randomization(labelList, originalDataSet):
    randomizedDataSet = []
    for i, do_flip in enumerate(labelList):    
        #if its a 1 that means the back half is lost
        x = originalDataSet[i] 
        if do_flip == 1:   
            x_rand = x[n_feat // 2:] + x[0:n_feat // 2]
        else:
            x_rand = x
        randomizedDataSet.append(x_rand)
    return randomizedDataSet

randomizedfeaturetrainingDataSet = randomization(randomizationSeedTrainingSet,featuretrainingDataSet)
randomizedfeaturetestDataSet = randomization(randomizationSeedTestSet, featuretestDataSet)
randomizedfeaturevalidationDataSet = randomization(randomizationSeedValidationSet,featurevalidationDataSet)

#     for i, do_flip in enumerate(randomizationSeedTrainingSet):    
#         #if its a 1 that means the back half is lost
#         if do_flip == 1:
#             x = featuretrainingDataSet[i]        
#             x_rand = x[n_feat // 2:] + x[0:n_feat // 2]
#         else:
#             x_rand = x
#         randomizedfeaturetrainingDataSet.append(x_rand)
            
print(randomizedfeaturetrainingDataSet[0])
print(randomizedfeaturetestDataSet[0])
print(randomizedfeaturevalidationDataSet[0])

['22', '1328', '22', '53', '2', '10', '16', '22', '10', '22', '8', '18', '9', '1104', '27', '58', '3', '14', '11', '18', '14', '24', '13', '23', '7', '1']
['18', '1157', '23', '59', '6', '14', '11', '18', '15', '22', '6', '20', '8', '1103', '25', '59', '7', '22', '15', '26', '14', '24', '10', '17', '13', '5']
['1104', '29', '63', '7', '23', '17', '19', '14', '20', '17', '8', '14', '9', '16', '1406', '18', '50', '7', '21', '11', '15', '14', '20', '9', '22', '2']


In [ ]:
print(featuretrainingDataSet[0])

In [ ]:
#pull labels out and then flip the respecitve lists within the data set corresponding to the data set
#save the array into np and then turn into torch
#implement logistical reggression

In [16]:

#converts to numpy array
x_np = np.array(featuretrainingDataSet[:100]).astype(np.float32)

# for i,x in enumerate(featuretrainingDataSet):
#     try:
#         np.array(x).astype(np.float32)
#     except:
#         print(i, x)

y_np = np.array(labeltrainingDataSet[:100])
# print(y_np.shape)

(100,)


In [28]:
print(type(featuretrainingDataSet_numpy[0][0]))

<class 'numpy.str_'>


In [17]:
#converts to pytorch array
x_pytorch = torch.from_numpy(x_np)
y_pytorch = torch.from_numpy(y_np)

In [ ]:
def train(model, loss, optimizer, x_val, y_val):
    x = Variable(x_val, requires_grad=False)
    y = Variable(y_val, requires_grad=False)
    
train()


In [11]:
array = [2,3,4]

qwe = "asdf"
def test(testArray, string):
    z = 0
    for x in testArray:
        z = z + x
        print(z)
    print(string)
print(array[0-2])
       
test(array, qwe)       


3
2
5
9
asdf
